In [2]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
from utils import add_Loss, clean_data
from tqdm.notebook import tqdm, trange

In [13]:
def unify_data(YEAR = 2019, SEASON = 'Kharif') :
    liste_dataframe = []
    monRepertoire = "RawData/"+str(YEAR)
    for f in tqdm(listdir(monRepertoire)) :
        if isfile(join(monRepertoire, f)) and (f[-11:-5] == SEASON or f[-9:-5] == SEASON):
            pathData = "RawData/"+str(YEAR)+"/"+f
            liste_dataframe.append(pd.read_excel(pathData))
    df = pd.concat(liste_dataframe)
    df.to_csv(f'RawDataUnified/RawData_{YEAR}_{SEASON}')

In [14]:
for YEAR in tqdm([2017,2018,2019]):
    for SEASON in ['Kharif', 'Rabi']:
        unify_data(YEAR,SEASON)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

In [10]:
df.save()

array([nan])

In [123]:
df=pd.read_csv("RawDataUnified/RawData_2019_Kharif")

def clean_data(df):
    """Clean les data pour l'année 2019"""
    #Suppression de la première colonne inutile (numérotation)
    df = df.drop(columns = ["Unnamed: 0"])
    #Suppression des colonnes sans valeur non nulle
    df = df.drop(columns = ["2018 Yield"])
    #Suppression des colonnes inutiles
    df = df.drop(columns = ["Season"])
    df = df.drop(columns = ["Cluster"])
    #Suppression des colonnes des infos administratives et definition de la colonne "key" conformément aux datasets de 03_pred
    df["Block"] = df["Block"].fillna("_")
    df["GP"] = df["GP"].fillna("_")
    df["Sub-District"] = df["Sub-District"].fillna("_")
    df["key"] = df["State"]+"_"+df["District"]+"_"+df["Sub-District"]+"_"+df["Block"]+"_"+df["GP"]
    df.key = df.key.astype(str).str.lower()
    df = df.drop(columns = ["State","District","Sub-District","Block","GP"])
    #On remplace les rendements nuls par leur moyenne
    for year in range(2000,2018):
        col = f"{year} Yield"
        df[col] = df[col].fillna(df[col].mean()) #A FAIRE vérifier que c'est pas abérant de faire ça pour l'anéee 2017. (df["2017 Yield"].isna().sum()) réponse : c'est bcp mais bon...
    for col in ["Area Sown (Ha)","Area Insured (Ha)","SI Per Ha (Inr/Ha)"]:
        df[col] = df[col].fillna(df[col].mean())
    return df

df=clean_data(df)

C:\Users\Amaury Lancelin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [124]:
df

,Crop,Area Sown (Ha),Area Insured (Ha),SI Per Ha (Inr/Ha),Sum Insured (Inr),Indemnity Level,2000 Yield,2001 Yield,2002 Yield,2003 Yield,...,2009 Yield,2010 Yield,2011 Yield,2012 Yield,2013 Yield,2014 Yield,2015 Yield,2016 Yield,2017 Yield,key
0,Arhar,340.012294,0.168514,28750.0,4844.772967,0.8,1527.033655,2337.250058,1095.885896,1351.243458,...,264.534667,150.820333,87.096222,108.775,70.00,295.00,89.00,99.000000,2021.056993,andhra pradesh_anantapur_agali___agali
1,Arhar,340.012294,0.168514,28750.0,4844.772967,0.8,1527.033655,2337.250058,1095.885896,1351.243458,...,264.534667,150.820333,87.096222,108.775,58.00,232.00,120.00,99.000000,2021.056993,andhra pradesh_anantapur_agali___akkagaladevar...
2,Arhar,340.012294,0.168514,28750.0,4844.772967,0.8,1527.033655,2337.250058,1095.885896,1351.243458,...,264.534667,150.820333,87.096222,108.775,67.00,295.00,89.00,99.000000,2021.056993,andhra pradesh_anantapur_agali___hulikeradevar...
3,Arhar,340.012294,0.168514,28750.0,4844.772967,0.8,1527.033655,2337.250058,1095.885896,1351.243458,...,264.534667,150.820333,87.096222,108.775,62.00,232.00,120.00,99.000000,2021.056993,andhra pradesh_anantapur_agali___inagalore
4,Arhar,340.012294,0.168514,28750.0,4844.772967,0.8,1527.033655,2337.250058,1095.885896,1351.243458,...,264.534667,150.820333,87.096222,108.775,70.00,295.00,89.00,99.000000,2021.056993,andhra pradesh_anantapur_agali___kodihalli
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372904,Aus Paddy,340.012294,1.857143,60500.0,112357.142857,0.9,1527.033655,2337.250058,1095.885896,1351.243458,...,1479.010000,1814.600000,1748.970000,1781.370,1942.86,2086.49,2346.79,2084.736928,2021.056993,west bengal_uttar dinajpur___hemtabad__
372905,Aus Paddy,340.012294,1.857143,60500.0,112357.142857,0.9,1527.033655,2337.250058,1095.885896,1351.243458,...,1479.010000,1814.600000,1748.970000,1781.370,1942.86,2086.49,2346.79,2084.736928,2021.056993,west bengal_uttar dinajpur___itahar__
372906,Aus Paddy,340.012294,1.857143,60500.0,112357.142857,0.9,1527.033655,2337.250058,1095.885896,1351.243458,...,1053.080000,1641.160000,1406.630000,1781.370,1942.86,2086.49,2346.79,2084.736928,2021.056993,west bengal_uttar dinajpur___kaliaganj__
372907,Aus Paddy,340.012294,1.857143,60500.0,112357.142857,0.9,1527.033655,2337.250058,1095.885896,1351.243458,...,1479.010000,1814.600000,1748.970000,1781.370,1942.86,2086.49,2346.79,2084.736928,2021.056993,west bengal_uttar dinajpur___karandighi__


In [125]:
add_Loss(df, year=2017)

(5, 372909)


,Crop,Area Sown (Ha),Area Insured (Ha),SI Per Ha (Inr/Ha),Sum Insured (Inr),Indemnity Level,2000 Yield,2001 Yield,2002 Yield,2003 Yield,...,2010 Yield,2011 Yield,2012 Yield,2013 Yield,2014 Yield,2015 Yield,2016 Yield,2017 Yield,key,Loss
0,Arhar,340.012294,0.168514,28750.0,4844.772967,0.8,1527.033655,2337.250058,1095.885896,1351.243458,...,150.820333,87.096222,108.775,70.00,295.00,89.00,99.000000,2021.056993,andhra pradesh_anantapur_agali___agali,12521.074567
1,Arhar,340.012294,0.168514,28750.0,4844.772967,0.8,1527.033655,2337.250058,1095.885896,1351.243458,...,150.820333,87.096222,108.775,58.00,232.00,120.00,99.000000,2021.056993,andhra pradesh_anantapur_agali___akkagaladevar...,12811.484033
2,Arhar,340.012294,0.168514,28750.0,4844.772967,0.8,1527.033655,2337.250058,1095.885896,1351.243458,...,150.820333,87.096222,108.775,67.00,295.00,89.00,99.000000,2021.056993,andhra pradesh_anantapur_agali___hulikeradevar...,12521.074567
3,Arhar,340.012294,0.168514,28750.0,4844.772967,0.8,1527.033655,2337.250058,1095.885896,1351.243458,...,150.820333,87.096222,108.775,62.00,232.00,120.00,99.000000,2021.056993,andhra pradesh_anantapur_agali___inagalore,12811.484033
4,Arhar,340.012294,0.168514,28750.0,4844.772967,0.8,1527.033655,2337.250058,1095.885896,1351.243458,...,150.820333,87.096222,108.775,70.00,295.00,89.00,99.000000,2021.056993,andhra pradesh_anantapur_agali___kodihalli,12521.074567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372904,Aus Paddy,340.012294,1.857143,60500.0,112357.142857,0.9,1527.033655,2337.250058,1095.885896,1351.243458,...,1814.600000,1748.970000,1781.370,1942.86,2086.49,2346.79,2084.736928,2021.056993,west bengal_uttar dinajpur___hemtabad__,0.000000
372905,Aus Paddy,340.012294,1.857143,60500.0,112357.142857,0.9,1527.033655,2337.250058,1095.885896,1351.243458,...,1814.600000,1748.970000,1781.370,1942.86,2086.49,2346.79,2084.736928,2021.056993,west bengal_uttar dinajpur___itahar__,0.000000
372906,Aus Paddy,340.012294,1.857143,60500.0,112357.142857,0.9,1527.033655,2337.250058,1095.885896,1351.243458,...,1641.160000,1406.630000,1781.370,1942.86,2086.49,2346.79,2084.736928,2021.056993,west bengal_uttar dinajpur___kaliaganj__,0.000000
372907,Aus Paddy,340.012294,1.857143,60500.0,112357.142857,0.9,1527.033655,2337.250058,1095.885896,1351.243458,...,1814.600000,1748.970000,1781.370,1942.86,2086.49,2346.79,2084.736928,2021.056993,west bengal_uttar dinajpur___karandighi__,0.000000


In [23]:
df.columns[0]

'Unnamed: 0'